In [ ]:
!pip install requests beautifulsoup4 netCDF4 geopandas shapely numpy
!pip install netCDF4 numpy
!pip install netCDF4 geopandas shapely

## Acess Key for CIROH s3 bucket

In [ ]:
## Will be provided upon request

import os
os.environ['AWS_ACCESS_KEY_ID'] = '----'
os.environ['AWS_SECRET_ACCESS_KEY'] = '-----'
os.environ['AWS_DEFAULT_REGION'] = '-----'


## List the contents 

In [ ]:
!aws s3 ls s3://ciroh-owp-hand-fim/

# Downloading the HAND Rasters and FIM Hydrofabric for Specific AOI for HAND FIM v-4.4

In [ ]:
!aws s3 sync s3://ciroh-owp-hand-fim/07080101 07080101

## Cloning the FIM script from github

In [ ]:
!git --version

!git clone https://github.com/NOAA-OWP/inundation-mapping.git inundation-mapping


## Creating the feature_id file from hydrotable

In [ ]:
import pandas as pd

# Load the hydrotable.csv file from AOI
hydrotable_path = r'07080101/hydrotable.csv'
hydrotable_df = pd.read_csv(hydrotable_path)

# Get unique feature IDs
unique_feature_ids = hydrotable_df['feature_id'].drop_duplicates()

# Create a DataFrame with the unique feature IDs
unique_feature_ids_df = pd.DataFrame(unique_feature_ids, columns=['feature_id'])

# Add a new column with row numbers starting from zero
unique_feature_ids_df.insert(0, '', range(len(unique_feature_ids_df)))

# Save the updated DataFrame to a new CSV file
output_csv_path = r'07080101_feature_id.csv'
unique_feature_ids_df.to_csv(output_csv_path, index=False)


#print(unique_feature_ids_df.head())
print('Row numbers added and CSV file saved successfully.')

In [ ]:
pip install xarray zarr fsspec s3fs

In [ ]:
import os
import pandas as pd
out_dir='07080101_run'
os.makedirs(out_dir,exist_ok=True)
fid_07080101=pd.read_csv('07080101_feature_id.csv')
print(fid_07080101)

## Short Range forecasting data for the AOI

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta
import netCDF4 as nc
import pandas as pd
import shutil

# Define directories
os.makedirs("Forecasted_streamflow", exist_ok=True)

url_base = "https://nomads.ncep.noaa.gov/pub/data/nccf/com/nwm/prod/"
output_dir = "07080101_run"
download_dir = "Forecasted_streamflow"
output_csv_filename = "07080101_feature_id.csv"
shutil.copy(output_csv_filename, output_dir)

def clear_download_directory(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

def download_nc_files(date_str, current_hour):
    url = f"{url_base}/nwm.{date_str}/short_range/"
    date_output_dir = os.path.join(download_dir, date_str)
    os.makedirs(date_output_dir, exist_ok=True)

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    pattern = re.compile(rf'nwm\.t{current_hour:02d}z\.short_range\.channel_rt\.f\d{{3}}\.conus\.nc')
    nc_files = [link['href'] for link in soup.find_all('a', href=True) if pattern.search(link['href'])]

    if not nc_files:
        return False, date_output_dir

    hour_output_dir = os.path.join(date_output_dir, f'{current_hour:02d}')
    os.makedirs(hour_output_dir, exist_ok=True)

    for nc_file in nc_files:
        file_url = url + nc_file
        file_path = os.path.join(hour_output_dir, nc_file)
        
        print(f'Downloading {file_url} to {file_path}')
        file_response = requests.get(file_url)
        with open(file_path, 'wb') as f:
            f.write(file_response.content)
        print(f'Download completed for {file_path}')

    print('All downloads completed for the date:', date_str)
    return True, hour_output_dir

def process_netcdf_file(netcdf_file_path, filter_df, output_folder_path):
    base_filename = os.path.basename(netcdf_file_path).replace('.nc', '')
    output_csv_file_path = os.path.join(output_folder_path, f'{base_filename}.csv')

    try:
        ds = nc.Dataset(netcdf_file_path, 'r')
        streamflow_data = ds.variables['streamflow'][:]
        feature_ids = ds.variables['feature_id'][:]
        ds.close()
    except Exception as e:
        print(f"Error reading NetCDF file {netcdf_file_path}: {e}")
        return

    if len(streamflow_data) == 0 or len(feature_ids) == 0:
        print(f"No data found in {netcdf_file_path}")
        return

    data_df = pd.DataFrame({
        'feature_id': feature_ids,
        'discharge': streamflow_data
    })

    filtered_df = data_df[data_df['feature_id'].isin(filter_df['feature_id'])]
    merged_df = pd.merge(filter_df[['feature_id']], filtered_df, on='feature_id')
    merged_df.to_csv(output_csv_file_path, index=False)
    print(f'Filtered DataFrame saved to {output_csv_file_path}')

def main():
    # Clear download directory before downloading new files
    clear_download_directory(download_dir)

    today = datetime.utcnow().strftime('%Y%m%d')
    current_hour = datetime.utcnow().hour

    success = False
    attempts = 0

    while not success and attempts < 24:
        attempts += 1
        success, date_output_dir = download_nc_files(today, current_hour)
        if not success:
            current_hour = (current_hour - 1) % 24
            if current_hour == 23:
                today = (datetime.utcnow() - timedelta(days=1)).strftime('%Y%m%d')

    if not success:
        print("No recent forecast data found. Exiting.")
        return
    
    filter_csv_file_path = os.path.join(output_dir, output_csv_filename)
    output_folder_path = os.path.join(download_dir, "Data")
    os.makedirs(output_folder_path, exist_ok=True)

    filter_df = pd.read_csv(filter_csv_file_path)

    if os.path.exists(date_output_dir):
        for root, _, files in os.walk(date_output_dir):
            for filename in files:
                if filename.endswith('.nc'):
                    netcdf_file_path = os.path.join(root, filename)
                    process_netcdf_file(netcdf_file_path, filter_df, output_folder_path)
    
    csv_directory = output_folder_path
    csv_files = [file for file in os.listdir(csv_directory) if file.endswith('.csv')]

    if not csv_files:
        print("No CSV files found after processing NetCDF files.")
        return

    combined_df = pd.concat([pd.read_csv(os.path.join(csv_directory, file))[['feature_id', 'discharge']] for file in csv_files])

    combined_df = combined_df.pivot_table(index='feature_id', values='discharge', aggfunc=list).apply(pd.Series.explode).reset_index()
    combined_df['discharge'] = combined_df['discharge'].astype(float)
    combined_df = combined_df.groupby('feature_id')['discharge'].apply(list).reset_index()
    for i in range(1, len(combined_df['discharge'][0]) + 1):
        combined_df[f'discharge_{i}'] = combined_df['discharge'].apply(lambda x: x[i-1] if i-1 < len(x) else None)
    combined_df.drop(columns=['discharge'], inplace=True)

    output_file = os.path.join(download_dir, "combined_streamflow.csv")
    combined_df.to_csv(output_file, index=False)

    # Extract maximum discharge for each feature_id
    max_discharge_df = combined_df.set_index('feature_id').max(axis=1).reset_index()
    max_discharge_df.columns = ['feature_id', 'discharge']

    max_output_file = os.path.join(download_dir, "forecasted_discharge.csv")
    max_discharge_df.to_csv(max_output_file, index=False)

    print(f'Maximum discharge values saved to {max_output_file}')

if __name__ == "__main__":
    main()


## Downloading NWM retrospective for all feature_ids to the directory with the new code

In [ ]:

#install these libraries if they aren't already installed
!pip install zarr
!pip install xarray
!pip install s3fs
!pip install numpy
 
import xarray as xr
import numpy as np
import s3fs
from datetime import datetime, timedelta

# open the zarr store
# url = "s3://noaa-nwm-retrospective-2-1-zarr-pds/chrtout.zarr"
url = "s3://noaa-nwm-retrospective-3-0-pds/CONUS/zarr/chrtout.zarr"
fs = s3fs.S3FileSystem(anon=True)

store = xr.open_zarr(s3fs.S3Map(url, s3=fs))

store
import pandas as pd
from datetime import datetime
import os

# Define the output directory
output_directory = '07080101_run'  # Change this to your desired directory

# Create the directory if it does not exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Function to get the time series for a specified reach id and time range
# then write it out to a csv file.
def GetAndWriteTimeSeriesAtReach(reach_id, start_time_index, end_time_index):
    # Filter the streamflow array for the specific reach_id and time range
    flows = streamflow_array.where(feature_id_array == reach_id, drop=True)
    df_flows = flows[start_time_index:end_time_index].to_dataframe()
    # Save the DataFrame to a CSV file
    csv_filename = os.path.join(output_directory, f'{reach_id}.csv')
    df_flows.to_csv(csv_filename)
    
    #print(f"Saved CSV file: {csv_filename}")

# Get the xarray array of the various values
time_array = store['time']
feature_id_array = store['feature_id']
streamflow_array = store['streamflow']

# Define the feature IDs to check for
feature_id_df = pd.read_csv('07080101_feature_id.csv')
feature_ids = feature_id_df['feature_id'].tolist()  # Assuming the column name is 'feature_id'

# Specify the start and end times of interest  24hr format
start_time = datetime(2015, 5, 23, 0, 0, 0)
end_time = datetime(2015, 5, 23, 4, 0, 0)

# Get the indices for the needed dates
zero_start_time = datetime(1979, 2, 1, 0, 0, 0)
start_time_index = int((start_time - zero_start_time).total_seconds() / 3600)
end_time_index = int((end_time - zero_start_time).total_seconds() / 3600)

# Loop over each feature ID from the CSV file, get the time series, and print it
for reach_id in feature_ids:
    GetAndWriteTimeSeriesAtReach(reach_id, start_time_index, end_time_index)


## Converting Streamflow data into HAND FIM input


In [1]:
import pandas as pd
import os

# List to store pivoted DataFrames
pivoted_dfs = []

output_directory = '07080101_run'

for filename in os.listdir(output_directory):
    if filename.endswith('.csv'):

        file_path = os.path.join(output_directory, filename)
        df = pd.read_csv(file_path)
        
        required_columns = {'time', 'feature_id', 'streamflow'}
        if required_columns.issubset(df.columns):
            # Keep only the 'time', 'feature_id', and 'streamflow' columns
            df = df[['time', 'feature_id', 'streamflow']]
            
            # Pivot the DataFrame
            df_pivoted = df.pivot(index='time', columns='feature_id', values='streamflow')
            
            # Remove the 'feature_id' header from the columns
            df_pivoted.columns.name = None
            
            # Append the pivoted DataFrame to the list
            pivoted_dfs.append(df_pivoted)
        else:
            # Print warning and skip this file
            print(f"Skipping {filename} - missing required columns: {required_columns - set(df.columns)}")


if pivoted_dfs:
    final_df = pd.concat(pivoted_dfs, axis=1)
    

    final_df = final_df.reset_index()
    print(final_df)
    output_file_path = os.path.join(output_directory,'concatenated_output.csv')
    final_df.to_csv(output_file_path, index=False)
     
    xyz_df = pd.read_csv(output_file_path)

    date_time_input = input("Enter the date and time in the format 'yyyy-mm-dd HH:MM:SS': ")

    date_time_index = xyz_df.index[xyz_df['time'] == date_time_input].tolist()

    if date_time_index:
        
        transposed_row = xyz_df.iloc[date_time_index[0]].transpose()
        transposed_row.name = date_time_input  # Set the name of the Series to the input date and time
        print(transposed_row)
    else:
        print("No matching date and time found.")
        
    transposed_row_df = transposed_row.to_frame().reset_index()
    transposed_row_df.columns = ['feature_id', 'discharge']  # Rename columns
    transposed_row_df = transposed_row_df.drop(index=0).reset_index(drop=True)  # Drop the 'time' row

    # Save the transposed DataFrame to a CSV file
    transposed_output_file_path = os.path.join(output_directory, '07080101_23may_2PM.csv')
    transposed_row_df.to_csv(transposed_output_file_path, index=False)

    print(f"Transposed output saved to {transposed_output_file_path}")

else:
    print("No valid files were processed.")


Skipping 07080101_23may.csv - missing required columns: {'time', 'streamflow'}
Skipping concatenated_output.csv - missing required columns: {'feature_id', 'streamflow'}
Skipping combined_flow.csv - missing required columns: {'time', 'feature_id', 'streamflow'}
Skipping 07080101_23may_2PM.csv - missing required columns: {'time', 'streamflow'}
                  time  14805783  14803879  14802695  14805583  14805111  \
0  2015-05-23 01:00:00      0.01      0.01      0.07      0.08       0.0   
1  2015-05-23 02:00:00      0.01      0.01      0.07      0.08       0.0   
2  2015-05-23 03:00:00      0.01      0.01      0.07      0.08       0.0   
3  2015-05-23 04:00:00      0.01      0.01      0.07      0.08       0.0   

      14806061  14806671  14804565  14803051  ...  14803713  14804179  \
0  1716.849962      0.01      0.84       0.0  ...       0.0       0.0   
1  1715.559962      0.01      0.83       0.0  ...       0.0       0.0   
2  1714.259962      0.01      0.83       0.0  ...       

Enter the date and time in the format 'yyyy-mm-dd HH:MM:SS':  2015-05-23 02:00:00


time        2015-05-23 02:00:00
14805783                   0.01
14803879                   0.01
14802695                   0.07
14805583                   0.08
                   ...         
14809463            1367.819969
14806221                   0.05
14806747                   0.09
14803569                   0.18
14805713                   0.01
Name: 2015-05-23 02:00:00, Length: 1195, dtype: object
Transposed output saved to 07080101_run/07080101_23may_2PM.csv


In [2]:
!pip install python-dotenv
!pip install numba
!pip install shapely pyogrio

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
  Using cached pyogrio-0.9.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
Using cached pyogrio-0.9.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (23.3 MB)


## Virtual .env file to replace docker

In [3]:
import os

print("Current Working Directory:", os.getcwd())

# Define the relative path to the .env file from the current working directory
env_file_path = "inundation-mapping/.env"  # Go up one level to the inundation-mapping directory

# Define the content for the .env file
env_content = """
inputsDir=inputs
outputsDir=output
"""
# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(env_file_path), exist_ok=True)

# Write the content to the .env file
with open(env_file_path, "w") as f:
    f.write(env_content)

print(f".env file created at {os.path.abspath(env_file_path)} with content:\n{env_content}")



Current Working Directory: /home/jovyan
.env file created at /home/jovyan/inundation-mapping/.env with content:

inputsDir=inputs
outputsDir=output



## FIM maps with retrospective discharge

In [4]:
import os
import sys
import shutil
import subprocess
import pandas as pd
from dotenv import load_dotenv
import warnings

# Suppress specific warnings
warnings.filterwarnings("ignore", message="invalid value encountered in cast", category=RuntimeWarning)

# Set up paths
repository_path = "/home/jovyan/inundation-mapping"  # Update to the correct repository path
tools_path = os.path.join(repository_path, "tools")
src_path = os.path.join(repository_path, "src")
output_path = "/home/jovyan/output"
fim_temp_path = "/home/jovyan/Outputs_temp"
data_path = "/home/jovyan/07080101_run"
huc_folder_path = "/home/jovyan/07080101"
output_huc_folder_path = os.path.join(output_path, "07080101")
branch_ids_file = os.path.join(output_huc_folder_path, "branch_ids.csv")
fim_inputs_file = os.path.join(output_path, "fim_inputs.csv")

# Ensure all necessary directories exist
os.makedirs(output_path, exist_ok=True)
os.makedirs(fim_temp_path, exist_ok=True)
os.makedirs(data_path, exist_ok=True)

# Step 1: Copy the entire HUC folder to the output directory
if os.path.exists(output_huc_folder_path):
    shutil.rmtree(output_huc_folder_path)  # Remove the existing folder if it exists
shutil.copytree(huc_folder_path, output_huc_folder_path)
print(f"Copied {huc_folder_path} to {output_huc_folder_path}")

# Step 2: Copy branch_ids.csv to the output folder, keep only the first row, and rename it to fim_inputs.csv
if os.path.exists(branch_ids_file):
    # Read the branch_ids.csv file
    df = pd.read_csv(branch_ids_file)
    
    # Keep only the first row
    #df_first_row = df.iloc[:0]
    
    # Save the first row to fim_inputs.csv
    #df_first_row.to_csv(fim_inputs_file, index=False)
    df.to_csv(fim_inputs_file, index=False)
    print(f"Copied first row of {branch_ids_file} to {fim_inputs_file}")
else:
    print(f"{branch_ids_file} does not exist")

# Change to the tools directory
os.chdir(tools_path)

# Load environment variables from .env file located in the inundation-mapping directory
dotenv_path = os.path.join(repository_path, ".env")
load_dotenv(dotenv_path)

# Verify that the environment variables are loaded
inputs_dir = os.getenv('inputsDir')
outputs_dir = os.getenv('outputsDir')
print(f"inputsDir: {inputs_dir}")
print(f"outputsDir: {outputs_dir}")

# Add src and repository_path to the Python path
sys.path.append(src_path)
sys.path.append(repository_path)

# Define the command to run the inundation script
command_to_run = [
    sys.executable,
    "inundate_mosaic_wrapper.py",
    "-y", output_path,
    "-u", "07080101",  # Replace with appropriate HUC
    "-f", os.path.join(data_path, "07080101_23may_2PM.csv"),
    "-i", os.path.join(output_huc_folder_path, "inundation_retro.tif"),
    "-d", os.path.join(output_huc_folder_path, "depth_retro.tif")
]

# Run the command with the correct working directory and PYTHONPATH
env = os.environ.copy()
env['PYTHONPATH'] = f"{src_path}{os.pathsep}{repository_path}"

result = subprocess.run(command_to_run, cwd=tools_path, env=env, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Print the output and error (if any)
print(result.stdout.decode())
if result.stderr:
    print(result.stderr.decode())

# Check if the command was successful
if result.returncode == 0:
    print("Inundation mapping completed successfully.")
else:
    print("Failed to complete inundation mapping.")


Copied /home/jovyan/07080101 to /home/jovyan/output/07080101
Copied first row of /home/jovyan/output/07080101/branch_ids.csv to /home/jovyan/output/fim_inputs.csv
inputsDir: inputs
outputsDir: output
Completed in 0.37 minutes.

Inundation mapping completed successfully.


## FIM maps with forecasted streamflow

In [ ]:
import os
import sys
import shutil
import subprocess
import pandas as pd
from dotenv import load_dotenv
import warnings

# Suppress specific warnings
warnings.filterwarnings("ignore", message="invalid value encountered in cast", category=RuntimeWarning)

# Set up paths
repository_path = "/home/jovyan/inundation-mapping"  # Update to the correct repository path
tools_path = os.path.join(repository_path, "tools")
src_path = os.path.join(repository_path, "src")
output_path = "/home/jovyan/output"
fim_temp_path = "/home/jovyan/Outputs_temp"
data_path = "/home/jovyan/Forecasted_streamflow"
huc_folder_path = "/home/jovyan/07080101"
output_huc_folder_path = os.path.join(output_path, "07080101")
branch_ids_file = os.path.join(output_huc_folder_path, "branch_ids.csv")
fim_inputs_file = os.path.join(output_path, "fim_inputs.csv")

# Ensure all necessary directories exist
os.makedirs(output_path, exist_ok=True)
os.makedirs(fim_temp_path, exist_ok=True)
os.makedirs(data_path, exist_ok=True)

# Step 1: Copy the entire HUC folder to the output directory
if os.path.exists(output_huc_folder_path):
    shutil.rmtree(output_huc_folder_path)  # Remove the existing folder if it exists
shutil.copytree(huc_folder_path, output_huc_folder_path)
print(f"Copied {huc_folder_path} to {output_huc_folder_path}")

# Step 2: Copy branch_ids.csv to the output folder, keep only the first row, and rename it to fim_inputs.csv
if os.path.exists(branch_ids_file):
    # Read the branch_ids.csv file
    df = pd.read_csv(branch_ids_file)
    
    # Keep only the first row
    #df_first_row = df.iloc[:0]
    
    # Save the first row to fim_inputs.csv
    #df_first_row.to_csv(fim_inputs_file, index=False)
    df.to_csv(fim_inputs_file, index=False)
    print(f"Copied first row of {branch_ids_file} to {fim_inputs_file}")
else:
    print(f"{branch_ids_file} does not exist")

# Change to the tools directory
os.chdir(tools_path)

# Load environment variables from .env file located in the inundation-mapping directory
dotenv_path = os.path.join(repository_path, ".env")
load_dotenv(dotenv_path)

# Verify that the environment variables are loaded
inputs_dir = os.getenv('inputsDir')
outputs_dir = os.getenv('outputsDir')
print(f"inputsDir: {inputs_dir}")
print(f"outputsDir: {outputs_dir}")

# Add src and repository_path to the Python path
sys.path.append(src_path)
sys.path.append(repository_path)

# Define the command to run the inundation script
command_to_run = [
    sys.executable,
    "inundate_mosaic_wrapper.py",
    "-y", output_path,
    "-u", "07080101",  # Replace with appropriate HUC
    "-f", os.path.join(data_path, "forecasted_discharge.csv"),
    "-i", os.path.join(output_huc_folder_path, "inundation_forecasted.tif"),
    "-d", os.path.join(output_huc_folder_path, "depth_forecasted.tif")
]

# Run the command with the correct working directory and PYTHONPATH
env = os.environ.copy()
env['PYTHONPATH'] = f"{src_path}{os.pathsep}{repository_path}"

result = subprocess.run(command_to_run, cwd=tools_path, env=env, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Print the output and error (if any)
print(result.stdout.decode())
if result.stderr:
    print(result.stderr.decode())

# Check if the command was successful
if result.returncode == 0:
    print("Inundation mapping completed successfully.")
else:
    print("Failed to complete inundation mapping.")
